source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path = "abfss://working-labs@adlsadataengdev.dfs.core.windows.net/bronze/geo-location/"

In [0]:
geoLocationSourceAPIURL = "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationSinkStorageAccountName = 'adlsadataengdev'
geoLocationSinkFolderName = 'geo-location'

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:
import requests
import pandas as pds
import json

In [0]:
geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponse)
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)

In [0]:
dailyPricingMarketNamesDF = spark.sql("SELECT MARKET_NAME FROM pricing_analytics.gold.reporting_dim_market_gold")

In [0]:
from pyspark.sql.types import *
marketNames = [dailyPricingMarketNames["MARKET_NAME"] for dailyPricingMarketNames in dailyPricingMarketNamesDF.collect()]

geoLocationAPIResponseList =[]

for marketName in marketNames:
    
    geoLocationSourceAPIURL = f"{geoLocationSourceAPIBaseURL}{marketName}{geoLocationSourceAPIURLOptions}"
    geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
    
    if isinstance(geoLocationAPIResponse, dict):
        geoLocationAPIResponseList.append(geoLocationAPIResponse)

print(geoLocationAPIResponse)
geoLocationSparkRDD = sc.parallelize(geoLocationAPIResponseList)
geoLocationSparkDF = spark.read.json(geoLocationSparkRDD)
(geoLocationSparkDF
 .filter("results.admin1 IS NOT NULL")
 .write
 .mode("overwrite")
 .json(geoLocationSinkFolderPath))


In [0]:
geoLocationBronzeDF = (spark
                       .read
                       .json(geoLocationSinkFolderPath)
                    )
display(geoLocationBronzeDF)